In [1]:
import src
from src.sweep0d import Sweep0D
from src.daq_driver import Daq
from src.base_sweep import BaseSweep
import nidaqmx
import time, os
import numpy as np
import qcodes as qc
from src.util import connect_to_station, connect_station_instruments, save_to_csv
from qcodes import initialise_or_create_database_at, load_by_run_spec
from qcodes import Station
from qcodes.instrument_drivers.Lakeshore.Model_372 import Model_372
from qcodes.instrument_drivers.american_magnetics.AMI430 import AMI430
from qcodes.instrument_drivers.stanford_research.SR860 import SR860
from qcodes_contrib_drivers.drivers.Oxford.IPS120 import OxfordInstruments_IPS120
from qcodes.instrument_drivers.tektronix.Keithley_2450 import Keithley2450

## Initialize the magnet driver

In [2]:
magnet = OxfordInstruments_IPS120('magnet', 'GPIB0::25::INSTR', use_gpib=True)

In [3]:
lockin = SR860('lockin', 'USB0::0xB506::0x2000::003594::INSTR')

Connected to: Stanford_Research_Systems SR860 (serial:003594, firmware:V1.51) in 0.06s


In [4]:
keith = Keithley2450('keith', 'USB0::0x05E6::0x2450::04358606::INSTR')

Connected to: KEITHLEY INSTRUMENTS 2450 (serial:04358606, firmware:1.6.4c) in 0.02s


In [5]:
daq = Daq('Daq', 'Dev1')

Connected to: NI DAQ USB-6215 (Dev1) in 0.72s


## Let's make sure we're connected and everything is as we expect

In [6]:
magnet.identify()

'IPS120-10  Version 3.07  (c) OXFORD 1996'

In [7]:
magnet.examine()

System Status: 
Normal
Activity: 
Hold
Local/Remote status: 
Local and unlocked
Switch heater: 
Off magnet at zero (switch closed)
Mode: 
Tesla, Magnet sweep: fast
Polarity: 
Desired: Positive, Magnet: Positive, Commanded: Positive, Both contactors open


## Set the Keithley output voltage

In [8]:
keith.output_enabled(1)

In [9]:
keith.source.voltage(0)

## Set the control status 
#### 0 : Local and locked
#### 1 : Remote and locked
#### 2 : Local and unlocked
#### 3 : Remote and unlocked

In [13]:
magnet.remote_status(3)

### We can check and set the field setpoints and the sweeprate of the field

In [13]:
magnet.field_setpoint()

1.0

In [11]:
magnet.field_setpoint(1)

In [12]:
magnet.sweeprate_field()

0.3

In [7]:
magnet.sweeprate_field(0.3)

## Create the sweep

In [14]:
sweep = Sweep0D(max_time=10000000)

In [16]:
daq.ai0.voltage.label = "Vxx_12-13"
daq.ai1.voltage.label = "Phase_12-13"
daq.ai2.voltage.label = "Vxy_12-16"
daq.ai3.voltage.label = "Phase_12-16"
lockin.X.label = "Isd_11-14"
lockin.P.label = "Phase_sd_11-14"
keith.sense.current.label = "I_leak"

In [17]:
sweep.follow_param(magnet.field, daq.ai0.voltage, daq.ai1.voltage, daq.ai2.voltage, daq.ai3.voltage, keith.sense.current, lockin.X, lockin.P)

## Setup the database

In [23]:
try:
    # Make sure database_name and the path are set to the correct values!
    db = "ctGXPS13.db"
    exp_name = "Rxx Rxy 1 - -1 T"
    sample_name = "T=3.5K_I11-14=100nA_Vg=0V_meas V_12-13_12-16"
    initialise_or_create_database_at('C:\\Users\\nanouser\\Documents\\MeasureIt\\Databases\\' + db)
    qc.new_experiment(exp_name, sample_name)
    if sweep is not None:
        sweep._create_measurement()
except:
    print("Error opening database")

## Check (and set) the activity status
#### 0 : Hold
#### 1 : Goto zero
#### 2 : Goto setpoint

In [50]:
magnet.activity()

'Hold'

## Turn the switch heater on to enable the field to be ramped

In [19]:
magnet.heater_on()

Setting switch heater... (wait 40s)


## Now, we can run to the field, either by calling 'run_to_field', or setting the activity to 'goto setpoint' 

In [24]:
magnet.run_to_field(-1)
%matplotlib qt
sweep.start()

In [ ]:
sweep.stop()

## If we want to stay at field to do measurements, hold the field in place by setting persistent mode

In [69]:
magnet.set_persistent()

Setting switch heater... (wait 40s)
Waiting for the switch heater to become superconducting


#### You can check that we're in persistent mode by checking the persistent field (field in the magnet) and the field (which is the current going through the leads- should be 0)

In [45]:
magnet.persistent_field()

6.0

In [46]:
magnet.field()

0.0

## Once done, leave persistent mode so we can ramp the field back to zero

In [70]:
magnet.leave_persistent_mode()

Setting switch heater... (wait 40s)


## Once we are done, we can go back to 0 either by 'run_to_field' or setting activity to 'goto zero'

In [45]:
magnet.run_to_field(0)

## Finally, switch the heater off and set us to 'hold'

In [46]:
magnet.heater_off()
magnet.activity(0)

Setting switch heater... (wait 40s)


## Double check to make sure the field and current are 0, and it is set to hold, and you can turn the IPS power off

In [78]:
sweep.stop()
magnet.activity(0)

In [71]:
sweep.kill()

In [79]:
sweep.save_data

True

In [80]:
sweep.is_running

False

In [82]:
directory = f'{os.environ["MeasureItHome"]}\\Origin Files\\'
ds = load_by_run_spec(
    experiment_name=exp_name,
    sample_name = sample_name
)
filename = f"{directory}\\{ds.run_id}_{ds.exp_name}_{ds.sample_name}.csv"
save_to_csv(ds, filename)